# Single source shortest path (MADlib v1.10+)
Given a graph and a source vertex, the single source shortest path (SSSP) algorithm finds a path from the source vertex to every other vertex in the graph, such that the sum of the weights of the path's edges is minimized.

In [9]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [10]:
%sql postgresql://gpdbchina@10.194.10.68:55000/madlib
#%sql postgresql://fmcquillan@localhost:5432/madlib
#%sql postgresql://gpadmin@54.197.30.46:10432/gpadmin

u'Connected: gpdbchina@madlib'

In [11]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.10.0-dev, git revision: v1.7.1-206-g7907a6b, cmake configuration time: Wed Jan 18 18:26:02 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create vertex and edge tables

In [84]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
        id INTEGER
        );

CREATE TABLE edge(
        src INTEGER,
        dest INTEGER,
        weight FLOAT8
        );

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(7);

INSERT INTO edge VALUES
(0, 1, 1.0),
(0, 2, 1.0),
(0, 4, 10.0),
(1, 2, 2.0),
(1, 3, 10.0),
(2, 3, 1.0),
(2, 5, 1.0),
(2, 6, 3.0),
(3, 0, 1.0),
(4, 0, -2.0),
(5, 6, 1.0),
(6, 7, 1.0);

Done.
Done.
Done.
8 rows affected.
12 rows affected.


[]

# 2.  Calculate the shortest paths from vertex 0

In [85]:
%%sql
DROP TABLE IF EXISTS out;

SELECT madlib.graph_sssp(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertix id column (NULL means use default naming)
                         'edge',        -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         0,             -- Source vertex for path calculation
                         'out');        -- Output table of shortest paths

SELECT * FROM out ORDER BY id;

Done.
1 rows affected.
8 rows affected.


id,weight,parent
0,0.0,0
1,1.0,0
2,1.0,0
3,2.0,2
4,10.0,0
5,2.0,2
6,3.0,5
7,4.0,6


# 3.  Get the shortest path to vertex 6

In [50]:
%%sql
SELECT madlib.graph_sssp_get_path('out',6) AS spath;

1 rows affected.


spath
"[0, 2, 5, 6]"


# 4.  Custom column names
i.e., not using defaults.  Create the vertex and edge tables:

In [86]:
%%sql 
DROP TABLE IF EXISTS vertex_alt,edge_alt;
CREATE TABLE vertex_alt AS SELECT id AS v_id FROM vertex;
CREATE TABLE edge_alt AS SELECT src AS e_src, dest, weight AS e_weight FROM edge;
SELECT * FROM edge_alt;

Done.
8 rows affected.
12 rows affected.
12 rows affected.


e_src,dest,e_weight
0,1,1.0
0,2,1.0
0,4,10.0
4,0,-2.0
2,3,1.0
2,5,1.0
2,6,3.0
6,7,1.0
3,0,1.0
1,2,2.0


# 5.0  Get the shortest path from vertex 1

In [87]:
%%sql
DROP TABLE IF EXISTS out_alt;

SELECT madlib.graph_sssp(
                         'vertex_alt',                  -- Vertex table
                         'v_id',                        -- Vertix id column (NULL means use default naming)
                         'edge_alt',                    -- Edge table
                         'src=e_src, weight=e_weight',  -- Edge arguments (NULL means use default naming)
                         1,                             -- Source vertex for path calculation
                         'out_alt');                    -- Output table of shortest paths

SELECT * FROM out_alt ORDER BY v_id;

Done.
1 rows affected.
8 rows affected.


v_id,e_weight,parent
0,4.0,3
1,0.0,1
2,2.0,1
3,3.0,2
4,14.0,0
5,3.0,2
6,4.0,5
7,5.0,6
